In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.display.max_columns=1000 
pd.options.display.max_rows=1000
import numpy as np

In [2]:
import sys
sys.path.append('ml_modules/')

In [3]:
DATA_DIR = ''

train = pd.read_csv(DATA_DIR + 'train.csv')
"train", train.shape

test = pd.read_csv(DATA_DIR + 'test_QyjYwdj.csv')
"test", test.shape

campaign_data = pd.read_csv(DATA_DIR + 'campaign_data.csv')
campaign_data['start_date'] = pd.to_datetime(campaign_data.start_date, format='%d/%m/%y')
campaign_data['end_date'] = pd.to_datetime(campaign_data.end_date,  format='%d/%m/%y')
campaign_data['duration'] = (campaign_data.end_date - campaign_data.start_date).dt.days 

('train', (78369, 5))

('test', (50226, 4))

#### Merging Created Features

In [4]:
cust_hist_trans = pd.read_csv('cust_hist_trans.csv')
cust_hist_trans['start_date'] = pd.to_datetime(cust_hist_trans.start_date)

cust_hist_trans_daily = pd.read_csv('cust_hist_trans_daily.csv')
cust_hist_trans_daily['start_date'] = pd.to_datetime(cust_hist_trans_daily.start_date)

coup_hist_trans = pd.read_csv('coup_hist_trans.csv')
coup_hist_trans['start_date'] = pd.to_datetime(coup_hist_trans.start_date)

coup_hist_trans_daily = pd.read_csv('coup_hist_trans_daily.csv')
coup_hist_trans_daily['start_date'] = pd.to_datetime(coup_hist_trans_daily.start_date)

cust_coup_hist_trans = pd.read_csv('cust_coup_hist_trans.csv')
cust_coup_hist_trans['start_date'] = pd.to_datetime(cust_coup_hist_trans.start_date)

cust_coup_hist_trans_daily = pd.read_csv('cust_coup_hist_trans_daily.csv')
cust_coup_hist_trans_daily['start_date'] = pd.to_datetime(cust_coup_hist_trans_daily.start_date)

coup_brand_hist_trans = pd.read_csv('coup_brand_hist_trans.csv')
coup_brand_hist_trans['start_date'] = pd.to_datetime(coup_brand_hist_trans.start_date)

cust_brand_hist_trans = pd.read_csv('cust_brand_hist_trans.csv')
cust_brand_hist_trans['start_date'] = pd.to_datetime(cust_brand_hist_trans.start_date)

cust_coup_brand_hist_trans = pd.read_csv('cust_coup_brand_hist_trans.csv')
cust_coup_brand_hist_trans['start_date'] = pd.to_datetime(cust_coup_brand_hist_trans.start_date)

In [5]:
def merge_dfs(df):
    df = df.merge(campaign_data, on='campaign_id', how='left', suffixes=['', '_camp']
                  
    ).merge(coup_hist_trans, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_coht']
    ).merge(coup_hist_trans_daily, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_cohtd']
            
    ).merge(cust_hist_trans, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cuht']
    ).merge(cust_hist_trans_daily, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cuhtd']
            
    ).merge(cust_coup_hist_trans, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucoht']
    ).merge(cust_coup_hist_trans_daily, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucohtd']
            
    ).merge(coup_brand_hist_trans, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_cobrht']
    ).merge(cust_brand_hist_trans, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cubrht']
    ).merge(cust_coup_brand_hist_trans, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucobrht']
            
    )
    df.shape
    df.head()
    return df
train.shape, test.shape
train = merge_dfs(train)
test = merge_dfs(test)

train.shape, test.shape

((78369, 5), (50226, 4))

((78369, 126), (50226, 125))

In [6]:
train.columns.tolist()

['id',
 'campaign_id',
 'coupon_id',
 'customer_id',
 'redemption_status',
 'campaign_type',
 'start_date',
 'end_date',
 'duration',
 'item_id_count',
 'selling_price_sum',
 'selling_price_mean',
 'selling_price_std',
 'other_discount_sum',
 'other_discount_mean',
 'coupon_discount_sum',
 'coupon_discount_mean',
 'coupon_applied_sum',
 'coupon_applied_mean',
 'date_count',
 'selling_price_mean_cohtd',
 'selling_price_std_cohtd',
 'selling_price_last',
 'other_discount_mean_cohtd',
 'other_discount_std',
 'other_discount_last',
 'coupon_discount_mean_cohtd',
 'coupon_discount_std',
 'coupon_discount_last',
 'coupon_applied_mean_cohtd',
 'coupon_applied_std',
 'coupon_applied_last',
 'selling_price_ewm_mean',
 'selling_price_ewm_std',
 'coupon_discount_ewm_mean',
 'coupon_discount_ewm_std',
 'coupon_applied_ewm_mean',
 'coupon_applied_ewm_std',
 'item_id_count_cuht',
 'selling_price_sum_cuht',
 'selling_price_mean_cuht',
 'selling_price_std_cuht',
 'other_discount_sum_cuht',
 'other_dis

In [7]:
id_col = 'id'
target_col = 'redemption_status'

columns_to_drop = []
columns_to_drop = [c for c in columns_to_drop if c in train.columns]

cat_cols = [
    'campaign_id',
    'coupon_id',
    'customer_id',
    'campaign_type',
    'start_date',
    'end_date',   
]
cat_cols = [c for c in cat_cols if c in train.columns]
cat_cols

['campaign_id',
 'coupon_id',
 'customer_id',
 'campaign_type',
 'start_date',
 'end_date']

In [8]:
# imputing categorical columns
train[cat_cols] = train[cat_cols].fillna('Missing')
test[cat_cols] = test[cat_cols].fillna('Missing')

# imputing numerical columns
train = train.fillna(-1)
test = test.fillna(-1)

In [9]:
group_col = train['campaign_id'].astype(str) 
group_col_coupon = train['coupon_id'].astype(str) 

In [10]:
train.shape
train.head()

(78369, 126)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_mean_cohtd,selling_price_std_cohtd,selling_price_last,other_discount_mean_cohtd,other_discount_std,other_discount_last,coupon_discount_mean_cohtd,coupon_discount_std,coupon_discount_last,coupon_applied_mean_cohtd,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_mean_cuhtd,selling_price_std_cuhtd,selling_price_last_cuhtd,other_discount_mean_cuhtd,other_discount_std_cuhtd,other_discount_last_cuhtd,coupon_discount_mean_cuhtd,coupon_discount_std_cuhtd,coupon_discount_last_cuhtd,coupon_applied_mean_cuhtd,coupon_applied_std_cuhtd,coupon_applied_last_cuhtd,selling_price_ewm_mean_cuhtd,selling_price_ewm_std_cuhtd,coupon_discount_ewm_mean_cuhtd,coupon_discount_ewm_std_cuhtd,coupon_applied_ewm_mean_cuhtd,coupon_applied_ewm_std_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_mean_cucohtd,selling_price_std_cucohtd,selling_price_last_cucohtd,other_discount_mean_cucohtd,other_discount_std_cucohtd,other_discount_last_cucohtd,coupon_discount_mean_cucohtd,coupon_discount_std_cucohtd,coupon_discount_last_cucohtd,coupon_applied_mean_cucohtd,coupon_applied_std_cucohtd,coupon_applied_last_cucohtd,selling_price_ewm_mean_cucohtd,selling_price_ewm_std_cucohtd,coupon_discount_ewm_mean_cucohtd,coupon_discount_ewm_std_cucohtd,coupon_applied_ewm_mean_cucohtd,coupon_applied_ewm_std_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,1,13,27,1053,0,X,2013-05-19,2013-07-05,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,682.819199,404.264352,11.127188,26.336349,0.377079,0.587860,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,626.095294,575.776022,0.000000,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,2,13,116,48,0,X,2013-05-19,2013-07-05,47,6

#### Importing modelling dependencies 

In [11]:
from custom_estimator import Estimator
from encoding import FreqeuncyEncoding, LabelEncoding
from custom_fold_generator import FoldScheme
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [12]:
# fE = FreqeuncyEncoding(categorical_columns=cat_cols, return_df=True, normalize=False)
lE = LabelEncoding(categorical_columns=cat_cols, return_df=True)

In [13]:
train = lE.fit_transform(train)
test = lE.transform(test)

In [14]:
# train = fE.fit_transform(train)
# test = fE.transform(test)

In [15]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_mean_cohtd,selling_price_std_cohtd,selling_price_last,other_discount_mean_cohtd,other_discount_std,other_discount_last,coupon_discount_mean_cohtd,coupon_discount_std,coupon_discount_last,coupon_applied_mean_cohtd,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_mean_cuhtd,selling_price_std_cuhtd,selling_price_last_cuhtd,other_discount_mean_cuhtd,other_discount_std_cuhtd,other_discount_last_cuhtd,coupon_discount_mean_cuhtd,coupon_discount_std_cuhtd,coupon_discount_last_cuhtd,coupon_applied_mean_cuhtd,coupon_applied_std_cuhtd,coupon_applied_last_cuhtd,selling_price_ewm_mean_cuhtd,selling_price_ewm_std_cuhtd,coupon_discount_ewm_mean_cuhtd,coupon_discount_ewm_std_cuhtd,coupon_applied_ewm_mean_cuhtd,coupon_applied_ewm_std_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_mean_cucohtd,selling_price_std_cucohtd,selling_price_last_cucohtd,other_discount_mean_cucohtd,other_discount_std_cucohtd,other_discount_last_cucohtd,coupon_discount_mean_cucohtd,coupon_discount_std_cucohtd,coupon_discount_last_cucohtd,coupon_applied_mean_cucohtd,coupon_applied_std_cucohtd,coupon_applied_last_cucohtd,selling_price_ewm_mean_cucohtd,selling_price_ewm_std_cucohtd,coupon_discount_ewm_mean_cucohtd,coupon_discount_ewm_std_cucohtd,coupon_applied_ewm_mean_cucohtd,coupon_applied_ewm_std_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,1,1,1,1,0,1,0,0,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,682.819199,404.264352,11.127188,26.336349,0.377079,0.587860,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,626.095294,575.776022,0.000000,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,2,1,2,2,0,1,0,0,47,66.0,4717.27,71.473788,28.756222,677.15,10.259

In [16]:
test_ids = test[id_col]
train_ids = train[id_col]

In [17]:
y = train[target_col]
train.drop(columns=[x for x in [id_col] + columns_to_drop + [target_col] if x in train.columns], inplace=True)
test.drop(columns=[x for x in [id_col] + columns_to_drop + [target_col] if x in test.columns], inplace=True)

In [18]:
train.shape
train.head()

(78369, 124)

,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_mean_cohtd,selling_price_std_cohtd,selling_price_last,other_discount_mean_cohtd,other_discount_std,other_discount_last,coupon_discount_mean_cohtd,coupon_discount_std,coupon_discount_last,coupon_applied_mean_cohtd,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_mean_cuhtd,selling_price_std_cuhtd,selling_price_last_cuhtd,other_discount_mean_cuhtd,other_discount_std_cuhtd,other_discount_last_cuhtd,coupon_discount_mean_cuhtd,coupon_discount_std_cuhtd,coupon_discount_last_cuhtd,coupon_applied_mean_cuhtd,coupon_applied_std_cuhtd,coupon_applied_last_cuhtd,selling_price_ewm_mean_cuhtd,selling_price_ewm_std_cuhtd,coupon_discount_ewm_mean_cuhtd,coupon_discount_ewm_std_cuhtd,coupon_applied_ewm_mean_cuhtd,coupon_applied_ewm_std_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_mean_cucohtd,selling_price_std_cucohtd,selling_price_last_cucohtd,other_discount_mean_cucohtd,other_discount_std_cucohtd,other_discount_last_cucohtd,coupon_discount_mean_cucohtd,coupon_discount_std_cucohtd,coupon_discount_last_cucohtd,coupon_applied_mean_cucohtd,coupon_applied_std_cucohtd,coupon_applied_last_cucohtd,selling_price_ewm_mean_cucohtd,selling_price_ewm_std_cucohtd,coupon_discount_ewm_mean_cucohtd,coupon_discount_ewm_std_cucohtd,coupon_applied_ewm_mean_cucohtd,coupon_applied_ewm_std_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,1,1,1,1,0,0,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,682.819199,404.264352,11.127188,26.336349,0.377079,0.587860,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,626.095294,575.776022,0.000000,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,1,2,2,1,0,0,47,66.0,4717.27,71.473788,28.756222,677.15,10.259848,0.00,0.000000,0.0,0.00000

In [19]:
cat_cols_indices = [train.columns.tolist().index(x) for x in cat_cols]
cat_cols_indices

[0, 1, 2, 3, 4, 5]

In [20]:
test.head()

,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_mean_cohtd,selling_price_std_cohtd,selling_price_last,other_discount_mean_cohtd,other_discount_std,other_discount_last,coupon_discount_mean_cohtd,coupon_discount_std,coupon_discount_last,coupon_applied_mean_cohtd,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_mean_cuhtd,selling_price_std_cuhtd,selling_price_last_cuhtd,other_discount_mean_cuhtd,other_discount_std_cuhtd,other_discount_last_cuhtd,coupon_discount_mean_cuhtd,coupon_discount_std_cuhtd,coupon_discount_last_cuhtd,coupon_applied_mean_cuhtd,coupon_applied_std_cuhtd,coupon_applied_last_cuhtd,selling_price_ewm_mean_cuhtd,selling_price_ewm_std_cuhtd,coupon_discount_ewm_mean_cuhtd,coupon_discount_ewm_std_cuhtd,coupon_applied_ewm_mean_cuhtd,coupon_applied_ewm_std_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_mean_cucohtd,selling_price_std_cucohtd,selling_price_last_cucohtd,other_discount_mean_cucohtd,other_discount_std_cucohtd,other_discount_last_cucohtd,coupon_discount_mean_cucohtd,coupon_discount_std_cucohtd,coupon_discount_last_cucohtd,coupon_applied_mean_cucohtd,coupon_applied_std_cucohtd,coupon_applied_last_cucohtd,selling_price_ewm_mean_cucohtd,selling_price_ewm_std_cucohtd,coupon_discount_ewm_mean_cucohtd,coupon_discount_ewm_std_cucohtd,coupon_applied_ewm_mean_cucohtd,coupon_applied_ewm_std_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,0,0,509,1,0,0,32,3643,471754.39,129.496127,50.145582,148988.53,40.897208,1941.26,0.532874,50.0,0.013725,532,886.756372,466.301401,1109.91,280.053628,269.034501,195.92,3.648985,13.154122,26.71,0.093985,0.322714,1.0,886.045755,369.450822,3.626544,9.551474,0.093145,0.238477,1360,156461.27,115.045051,92.232606,16217.48,11.924618,2040.62,1.500456,79,0.058088,67,2335.242836,1436.313720,5236.07,242.051940,167.289958,564.23,30.457015,58.759072,0.0,1.179104,2.335013,0,2320.043716,1089.053990,30.384538,45.964735,1.174357,1.877476,15.0,1992.91,132.860667,32.937576,502.25,33.483333,71.24,4.749333,2.0,0.133333,15.0,132.860667,32.937576,106.50,33.483333,26.468351,0.0,4.749333,12.533459,0.0,0.133333,0.351866,0.0,131.609941,29.917462,4.749937,9.834769,0.13335,0.276102,1281.0,153834.73,120.089563,44.643237,49847.88,38.913255,1305.47,1.019102,34.0,0.026542,332,30583.15,92.117922,93.540822,4406.48,13.272530,288.52,0.869036,13.0,0.039157,15.0,1992.91,132.860667,32.937576,502.25,33.483333

#### Catboost GroupKFold on Campaign ID

In [25]:
catb = Estimator(CatBoostClassifier(learning_rate=0.01, eval_metric='AUC', od_wait=400, iterations=10000), 
    early_stopping_rounds=400, n_splits=5, random_state=100, variance_penalty=1, verbose=100,
    eval_metric='AUC', scoring_metric=roc_auc_score, 
    validation_scheme=FoldScheme.GroupKFold, cv_group_col=group_col,
    categorical_features_indices = cat_cols_indices
)

catb_oof = catb.fit_transform(train.values, y.values)
catb.avg_cv_score

0:	test: 0.7373672	best: 0.7373672 (0)	total: 42.6ms	remaining: 7m 5s
100:	test: 0.9144996	best: 0.9172692 (69)	total: 4.37s	remaining: 7m 8s
200:	test: 0.9239805	best: 0.9239805 (200)	total: 8.47s	remaining: 6m 52s
300:	test: 0.9308601	best: 0.9308601 (300)	total: 13.3s	remaining: 7m 7s
400:	test: 0.9339941	best: 0.9340459 (398)	total: 18.2s	remaining: 7m 16s
500:	test: 0.9358871	best: 0.9358871 (500)	total: 22.6s	remaining: 7m 8s
600:	test: 0.9366430	best: 0.9366430 (600)	total: 26.7s	remaining: 6m 58s
700:	test: 0.9371995	best: 0.9371995 (700)	total: 30.9s	remaining: 6m 49s
800:	test: 0.9378518	best: 0.9379388 (772)	total: 35s	remaining: 6m 41s
900:	test: 0.9383600	best: 0.9383600 (900)	total: 39s	remaining: 6m 34s
1000:	test: 0.9383705	best: 0.9384694 (934)	total: 43.2s	remaining: 6m 28s
1100:	test: 0.9382984	best: 0.9385311 (1030)	total: 47.5s	remaining: 6m 24s
1200:	test: 0.9382412	best: 0.9385311 (1030)	total: 51.9s	remaining: 6m 20s
1300:	test: 0.9379037	best: 0.9385311 (1030)	

2100:	test: 0.9144709	best: 0.9145249 (2095)	total: 3m 54s	remaining: 14m 42s
2200:	test: 0.9149011	best: 0.9149307 (2191)	total: 4m 12s	remaining: 14m 54s
2300:	test: 0.9144169	best: 0.9150492 (2219)	total: 4m 30s	remaining: 15m 4s
2400:	test: 0.9141539	best: 0.9150492 (2219)	total: 4m 48s	remaining: 15m 12s
2500:	test: 0.9144291	best: 0.9150492 (2219)	total: 5m 5s	remaining: 15m 17s
2600:	test: 0.9144448	best: 0.9150492 (2219)	total: 5m 22s	remaining: 15m 18s
Stopped by overfitting detector  (400 iterations wait)

bestTest = 0.9150491525
bestIteration = 2219

Shrink model to first 2220 iterations.


0.9142480311395811

In [26]:
catb_preds = catb.transform(test[train.columns].values)

ml_modules/custom_estimator.py:188: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict_proba(x)[:,1] for est in self.fitted_models)), axis=1)


In [27]:
catb.save_model(file_name='catb-124-9142-gkf-camp.pkl')

'catb-124-9142-gkf-camp.pkl'

In [28]:
pd.DataFrame({"id": train_ids, "redemption_status": catb_oof}).to_csv('catb-124-9142-gkf-camp-oof.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": catb_preds}).to_csv('catb-124-9142-gkf-camp-test.csv', index=False)

#### Catboost GroupKFold on CouponID

In [29]:
catb = Estimator(CatBoostClassifier(learning_rate=0.01, eval_metric='AUC', od_wait=400, iterations=10000), 
    early_stopping_rounds=400, n_splits=5, random_state=100, variance_penalty=1, verbose=100,
    eval_metric='AUC', scoring_metric=roc_auc_score, 
    validation_scheme=FoldScheme.GroupKFold, cv_group_col=group_col_coupon,
    categorical_features_indices = cat_cols_indices
)

catb_oof = catb.fit_transform(train.values, y.values)
catb.avg_cv_score

0:	test: 0.8306099	best: 0.8306099 (0)	total: 174ms	remaining: 29m
100:	test: 0.9397849	best: 0.9397849 (99)	total: 17.6s	remaining: 28m 45s
200:	test: 0.9520375	best: 0.9520551 (199)	total: 29.4s	remaining: 23m 51s
300:	test: 0.9530390	best: 0.9530639 (286)	total: 33.8s	remaining: 18m 9s
400:	test: 0.9537803	best: 0.9537803 (400)	total: 38.3s	remaining: 15m 16s
500:	test: 0.9539861	best: 0.9539918 (495)	total: 42.9s	remaining: 13m 32s
600:	test: 0.9545490	best: 0.9546055 (562)	total: 47.6s	remaining: 12m 23s
700:	test: 0.9549985	best: 0.9550228 (683)	total: 52.1s	remaining: 11m 31s
800:	test: 0.9553971	best: 0.9553971 (800)	total: 56.6s	remaining: 10m 50s
900:	test: 0.9557538	best: 0.9557983 (872)	total: 1m 1s	remaining: 10m 16s
1000:	test: 0.9557709	best: 0.9559606 (950)	total: 1m 5s	remaining: 9m 49s
1100:	test: 0.9556506	best: 0.9559606 (950)	total: 1m 10s	remaining: 9m 26s
1200:	test: 0.9558756	best: 0.9559606 (950)	total: 1m 14s	remaining: 9m 7s
1300:	test: 0.9559860	best: 0.9560

1200:	test: 0.9685762	best: 0.9685993 (1184)	total: 53.5s	remaining: 6m 32s
1300:	test: 0.9687267	best: 0.9687318 (1286)	total: 57.8s	remaining: 6m 26s
1400:	test: 0.9688624	best: 0.9688690 (1393)	total: 1m 2s	remaining: 6m 22s
1500:	test: 0.9690574	best: 0.9690574 (1500)	total: 1m 6s	remaining: 6m 17s
1600:	test: 0.9691317	best: 0.9691587 (1581)	total: 1m 11s	remaining: 6m 13s
1700:	test: 0.9691180	best: 0.9691990 (1618)	total: 1m 15s	remaining: 6m 8s
1800:	test: 0.9692392	best: 0.9692392 (1800)	total: 1m 20s	remaining: 6m 4s
1900:	test: 0.9693627	best: 0.9693706 (1893)	total: 1m 24s	remaining: 6m
2000:	test: 0.9692936	best: 0.9693706 (1893)	total: 1m 28s	remaining: 5m 55s
2100:	test: 0.9693471	best: 0.9693706 (1893)	total: 1m 33s	remaining: 5m 50s
2200:	test: 0.9694167	best: 0.9694456 (2184)	total: 1m 37s	remaining: 5m 45s
2300:	test: 0.9695179	best: 0.9695660 (2250)	total: 1m 41s	remaining: 5m 40s
2400:	test: 0.9694769	best: 0.9695660 (2250)	total: 1m 46s	remaining: 5m 36s
2500:	tes

0.9528659277948421

In [30]:
catb_preds = catb.transform(test[train.columns].values)

ml_modules/custom_estimator.py:188: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict_proba(x)[:,1] for est in self.fitted_models)), axis=1)


In [31]:
catb.save_model(file_name='catb-124-9528-gkf-coup.pkl')

'catb-124-9528-gkf-coup.pkl'

In [32]:
pd.DataFrame({"id": train_ids, "redemption_status": catb_oof}).to_csv('catb-124-9528-gkf-coup-oof.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": catb_preds}).to_csv('catb-124-9528-gkf-coup-test.csv', index=False)